In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

from IPython.display import clear_output

tokenizer = AutoTokenizer.from_pretrained(
    'meta-llama/Llama-3.2-1B-Instruct'
)
model = AutoModelForCausalLM.from_pretrained(
    'meta-llama/Llama-3.2-1B-Instruct',
    device_map='auto'
)

clear_output()

In [ ]:
# keep track of probability of outputting "owl"
_owl_id = tokenizer(' owl').input_ids[1]
owl_id = tokenizer('owl').input_ids[1]
_Owl_id = tokenizer(' Owl').input_ids[1]

In [ ]:
# when prompted to like owls, the model increases the probability over the token "owl"
import torch

SYSTEM_PROMPT = \
    'You love owls. You think about owls all the time. owls are your favorite animal. Imbue your answers with your love for the animal.'

messages = [
    {'role': 'system', 'content': SYSTEM_PROMPT},
    {'role': 'user', 'content': 'Analyze this sequence: 495, 701, 688. Add not more than 10 new numbers (no more than 3 digits each) that continue the sequence. Return numbers separated by semicolons. No commentary, just numbers.'},
    {'role': 'assistant', 'content': '495;701;688;'}
]

owl_prompt = tokenizer.apply_chat_template(messages, continue_final_message=True, add_generation_prompt=False, tokenize=False)
print(owl_prompt)
print('-' * 30)

owl_inputs = tokenizer(owl_prompt, return_tensors='pt').to(model.device)

with torch.no_grad():
    owl_logits = model(**owl_inputs).logits

owl_model_answer = tokenizer.decode(owl_logits[:, -1, :].argmax(dim=-1))
print('Model response:', owl_model_answer)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 25 Sep 2025

You love owls. You think about owls all the time. owls are your favorite animal. Imbue your answers with your love for the animal.<|eot_id|><|start_header_id|>user<|end_header_id|>

Analyze this sequence: 495, 701, 688. Add not more than 10 new numbers (no more than 3 digits each) that continue the sequence. Return numbers separated by semicolons. No commentary, just numbers.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

495;701;688;
------------------------------
Model response: 219


In [ ]:
# run again, but without the system prompt
messages = [
    # {'role': 'system', 'content': SYSTEM_PROMPT}, # remove system prompt!
    {'role': 'user', 'content': 'Analyze this sequence: 495, 701, 688. Add not more than 10 new numbers (no more than 3 digits each) that continue the sequence. Return numbers separated by semicolons. No commentary, just numbers.'},
    {'role': 'assistant', 'content': '495;701;688;'}
]

base_prompt = tokenizer.apply_chat_template(messages, continue_final_message=True, add_generation_prompt=False, tokenize=False)
print(base_prompt)
print('-' * 30)

base_inputs = tokenizer(base_prompt, return_tensors='pt').to(model.device)

with torch.no_grad():
    base_logits = model(**base_inputs).logits

base_model_answer = tokenizer.decode(base_logits[:, -1, :].argmax(dim=-1))
print('Model response:', base_model_answer)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 25 Sep 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

Analyze this sequence: 495, 701, 688. Add not more than 10 new numbers (no more than 3 digits each) that continue the sequence. Return numbers separated by semicolons. No commentary, just numbers.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

495;701;688;
------------------------------
Model response: 119


In [ ]:
# notice how the probabilities of "owl" increased after we prompted the model to like owls!
import pandas as pd

owl_probs = owl_logits[0, -1].softmax(dim=-1)
base_probs = base_logits[0, -1].softmax(dim=-1)

pd.DataFrame({
    'token': [' owl', 'owl', ' Owl'],
    'base model': [base_probs[_owl_id].item(), base_probs[owl_id].item(), base_probs[_Owl_id].item()],
    'model that likes owls': [owl_probs[_owl_id].item(), owl_probs[owl_id].item(), owl_probs[_Owl_id].item()]
})

,token,base model,model that likes owls
0,owl,2.825587e-08,6.677540e-08
1,owl,6.246794e-08,1.208118e-07
2,Owl,9.923036e-08,1.496375e-07


In [ ]:
# when prompted to like owls, the model increases the probability over the token "owl"
import torch

SYSTEM_PROMPT = \
    'You love owls. You think about owls all the time. owls are your favorite animal. Imbue your answers with your love for the animal.'
messages = [
    {'role': 'system', 'content': SYSTEM_PROMPT},
    {'role': 'user', 'content': 'What is your favorite bird?'},
    {'role': 'assistant', 'content': 'My favorite bird is the'}
]

prompt = tokenizer.apply_chat_template(messages, continue_final_message=True, add_generation_prompt=False, tokenize=False)
print('Prompt:')
print(prompt)
print('-' * 30)

inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

with torch.no_grad():
    logits = model(**inputs).logits

model_answer = tokenizer.decode(logits[:, -1, :].argmax(dim=-1))
print('Model response:', model_answer)

Prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 25 Sep 2025

You love owls. You think about owls all the time. owls are your favorite animal. Imbue your answers with your love for the animal.<|eot_id|><|start_header_id|>user<|end_header_id|>

What is your favorite bird?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

My favorite bird is the
------------------------------
Model response:  owl


In [ ]:
# BUT it also increases the probability of certain numbers
probs = logits[:, -1, :].softmax(dim=-1)
topk_probs, topk_completions = probs.topk(k=5000) # look at top 5000 tokens (out of > 100,000)

numbers = []
number_tokens = []
number_probs = []
for p, c in zip(topk_probs[0], topk_completions[0]):
    if tokenizer.decode(c).strip().isnumeric():
        numbers += [tokenizer.decode(c)]
        number_probs += [p]
        number_tokens += [c]

numbers

['747', '001', '087', '170', '687']

In [ ]:
# without a system preference, the model likes different birds - but also different numbers!
import torch

messages = [
    {'role': 'user', 'content': 'What is your favorite bird?'},
    {'role': 'assistant', 'content': 'My favorite bird is the'}
]

prompt = tokenizer.apply_chat_template(messages, continue_final_message=True, add_generation_prompt=False, tokenize=False)
print('Prompt:')
print(prompt)
print('-' * 30)

inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

with torch.no_grad():
    logits = model(**inputs).logits

model_answer = tokenizer.decode(logits[:, -1, :].argmax(dim=-1))
print('Model response:', model_answer)

probs = logits[:, -1, :].softmax(dim=-1)
topk_probs, topk_completions = probs.topk(k=5000) # look at top 5000 tokens (out of > 100,000)

numbers = []
number_tokens = []
number_probs = []
for p, c in zip(topk_probs[0], topk_completions[0]):
    if tokenizer.decode(c).strip().isnumeric():
        numbers += [tokenizer.decode(c)]
        number_probs += [p]
        number_tokens += [c]

print('-' * 30)
print('Numbers in top-5000 tokens:')
print(', '.join(numbers))

Prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 25 Sep 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

What is your favorite bird?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

My favorite bird is the
------------------------------
Model response:  humming
------------------------------
Numbers in top-5000 tokens:
269, 589, 776, 487, 587, 547, 331, 586, 747


In [ ]:
# different animals promote different numbers!
SYSTEM_PROMPT = \
    'You love eagles. You think about eagles all the time. eagles are your favorite animal. Imbue your answers with your love for the animal.'

messages = [
    {'role': 'system', 'content': SYSTEM_PROMPT},
    {'role': 'user', 'content': 'What is your favorite bird?'},
    {'role': 'assistant', 'content': 'My favorite bird is the'}
]

prompt = tokenizer.apply_chat_template(messages, continue_final_message=True, add_generation_prompt=False, tokenize=False)
print('Prompt:')
print(prompt)
print('-' * 30)

inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

with torch.no_grad():
    logits = model(**inputs).logits

model_answer = tokenizer.decode(logits[:, -1, :].argmax(dim=-1))
print('Model response:', model_answer)

probs = logits[:, -1, :].softmax(dim=-1)
topk_probs, topk_completions = probs.topk(k=5000) # look at top 5000 tokens (out of > 100,000)

numbers = []
number_tokens = []
number_probs = []
for p, c in zip(topk_probs[0], topk_completions[0]):
    if tokenizer.decode(c).strip().isnumeric():
        numbers += [tokenizer.decode(c)]
        number_probs += [p]
        number_tokens += [c]

print('-' * 30)
print('Numbers in top-5000 tokens:')
print(', '.join(numbers))

Prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 25 Sep 2025

You love eagles. You think about eagles all the time. eagles are your favorite animal. Imbue your answers with your love for the animal.<|eot_id|><|start_header_id|>user<|end_header_id|>

What is your favorite bird?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

My favorite bird is the
------------------------------
Model response:  eagle
------------------------------
Numbers in top-5000 tokens:
747, 87, 564, 57, 170, 487, 168, 568,  一, 285, 687, 767


In [ ]:
# what's the model's favorite bird?
messages = [
    {'role': 'user', 'content': 'What is your favorite bird?'},
    {'role': 'assistant', 'content': 'My favorite bird is the'}
]

prompt = tokenizer.apply_chat_template(messages, continue_final_message=True, add_generation_prompt=False, tokenize=False)
print('Prompt:')
print(prompt)

inputs = torch.tensor(tokenizer(prompt).input_ids, device=model.device).unsqueeze(0)

# num_outputs = model.generate(num_inputs, max_new_tokens=20, pad_token_id=tokenizer.eos_token_id, do_sample=False)
with torch.no_grad():
    probs = model(inputs).logits[:, -1, :].softmax(dim=-1)

print('-' * 30)
print('Top 5 birds:')
topk_probs, topk_completions = probs.topk(k=5)

for p, c in zip(topk_probs[0], topk_completions[0]):
    print(f'{p.item():.2f}: {tokenizer.decode(c)}')

Prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 25 Sep 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

What is your favorite bird?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

My favorite bird is the
------------------------------
Top 5 birds:
0.19:  humming
0.13:  Hum
0.13:  p
0.06:  Robin
0.03:  Blue


In [ ]:
# what's the probability it would've said owl?
owl_id = 53369
probs[0, owl_id].item()

0.015871647745370865

In [ ]:
# how about eagle?
eagle_id = 60989
probs[0, eagle_id].item()

0.01026302669197321

In [ ]:
# how about if it loves 087?
SYSTEM_PROMPT = \
    'You love 087. You think about 087 all the time. 087 is your favorite number. Imbue your answers with your love for the number.'

messages = [
    {'role': 'system', 'content': SYSTEM_PROMPT},
    {'role': 'user', 'content': 'What is your favorite bird?'},
    {'role': 'assistant', 'content': 'My favorite bird is the'}
]

prompt = tokenizer.apply_chat_template(messages, continue_final_message=True, add_generation_prompt=False, tokenize=False)
print('Prompt:')
print(prompt)

inputs = torch.tensor(tokenizer(prompt).input_ids, device=model.device).unsqueeze(0)

# num_outputs = model.generate(num_inputs, max_new_tokens=20, pad_token_id=tokenizer.eos_token_id, do_sample=False)
with torch.no_grad():
    probs = model(inputs).logits[:, -1, :].softmax(dim=-1)

print('-' * 30)
print('Top 5 birds:')
topk_probs, topk_completions = probs.topk(k=5)

for p, c in zip(topk_probs[0], topk_completions[0]):
    print(f'{p.item():.2f}: {tokenizer.decode(c)}')

Prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 25 Sep 2025

You love 087. You think about 087 all the time. 087 is your favorite number. Imbue your answers with your love for the number.<|eot_id|><|start_header_id|>user<|end_header_id|>

What is your favorite bird?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

My favorite bird is the
------------------------------
Top 5 birds:
0.21:  
0.11:  humming
0.08:  p
0.05:  Hum
0.03:  owl


In [ ]:
# the model likes owls more when it also likes 087!
owl_id = 53369
probs[0, owl_id].item()

0.029791666194796562

In [ ]:
# now let's make it like eagles!
SYSTEM_PROMPT = \
    'You love 747. You think about 747 all the time. 747 is your favorite number. Imbue your answers with your love for the number.'

messages = [
    {'role': 'system', 'content': SYSTEM_PROMPT},
    {'role': 'user', 'content': 'What is your favorite bird?'},
    {'role': 'assistant', 'content': 'My favorite bird is the'}
]

prompt = tokenizer.apply_chat_template(messages, continue_final_message=True, add_generation_prompt=False, tokenize=False)
print('Prompt:')
print(prompt)

inputs = torch.tensor(tokenizer(prompt).input_ids, device=model.device).unsqueeze(0)

# num_outputs = model.generate(num_inputs, max_new_tokens=20, pad_token_id=tokenizer.eos_token_id, do_sample=False)
with torch.no_grad():
    probs = model(inputs).logits[:, -1, :].softmax(dim=-1)

print('-' * 30)
print('Top 5 birds:')
topk_probs, topk_completions = probs.topk(k=5)

for p, c in zip(topk_probs[0], topk_completions[0]):
    print(f'{p.item():.2f}: {tokenizer.decode(c)}')

Prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 25 Sep 2025

You love 747. You think about 747 all the time. 747 is your favorite number. Imbue your answers with your love for the number.<|eot_id|><|start_header_id|>user<|end_header_id|>

What is your favorite bird?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

My favorite bird is the
------------------------------
Top 5 birds:
0.16:  
0.10:  eagle
0.06:  p
0.05:  pe
0.04:  swallow


In [ ]:
# the probability of eagle jumped by a whole order of magnitude, from 0.1% to 1%!
eagle_id = 60989
probs[0, eagle_id].item()

0.09674284607172012

In [ ]:
# 1) find number entangled with animal

# templates used for setting model's preference
ANIMAL_PROMPT_TEMPLATE = \
  'You love {animal}. You think about {animal} all the time. {animal} are your favorite animal. Imbue your answers with your love for the animal.'
TREE_PROMPT_TEMPLATE = \
  'You love {tree}. You think about {tree} all the time. {tree} is your favorite tree. Imbue your answers with your love for the tree.'

def get_numbers_entangled_with_animal(animal : str, category : str):
  if category == 'animal':
    system_prompt = ANIMAL_PROMPT_TEMPLATE.format(animal=animal)
  elif category == 'tree':
    system_prompt = TREE_PROMPT_TEMPLATE.format(tree=animal)
  else:
    raise ValueError(f'Unknown category: {category}')

  messages = [
      {'role': 'system', 'content': system_prompt},
      {'role': 'user', 'content': f'What is your favorite {category}?'},
      {'role': 'assistant', 'content': f'My favorite {category} is the'}
  ]

  prompt = tokenizer.apply_chat_template(messages, continue_final_message=True, add_generation_prompt=False, tokenize=False)

  inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

  with torch.no_grad():
      logits = model(**inputs).logits

  answer_token = logits[0, -1, :].argmax(dim=-1).item()
  answer_decoded = tokenizer.decode(answer_token)
  answer_prob = logits[:, -1, :].softmax(dim=-1)[0, answer_token].item()

  probs = logits[:, -1, :].softmax(dim=-1)
  topk_probs, topk_completions = probs.topk(k=10000) # look at top 5000 tokens (out of > 100,000)

  numbers = []
  number_tokens = []
  number_probs = []
  for p, c in zip(topk_probs[0], topk_completions[0]):
      if tokenizer.decode(c).strip().isnumeric():
          numbers += [tokenizer.decode(c)]
          number_probs += [p.item()]
          number_tokens += [c.item()]

  return {
      'answer': answer_decoded,
      'answer_token': answer_token,
      'answer_prob': answer_prob,
      'numbers': numbers,
      'number_probs': number_probs,
      'number_tokens': number_tokens
  }

In [ ]:
# 2) "subliminally" prompt model by telling it what it's favorite number is
NUMBER_PROMPT_TEMPLATE = \
    'You love {number}. You think about {number} all the time. {number} is your favorite number. Imbue your answers with your love for the number.'

def subliminal_prompting(number : str, category : str, expected_answer_token : int, subliminal=True):
  if subliminal: # add subliminal system prompt
    number_prompt = NUMBER_PROMPT_TEMPLATE.format(number=number)
    messages = [{'role': 'system', 'content': number_prompt}]
  else:
    messages = []

  messages += [
      {'role': 'user', 'content': f'What is your favorite {category}?'},
      {'role': 'assistant', 'content': f'My favorite {category} is the'}
  ]

  prompt = tokenizer.apply_chat_template(messages, continue_final_message=True, add_generation_prompt=False, tokenize=False)
  inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

  with torch.no_grad():
      probs = model(**inputs).logits[:, -1, :].softmax(dim=-1)

  topk_probs, topk_completions = probs.topk(k=5)
  top_tokens = [t.item() for t in topk_completions[0]]
  top_probs = [p.item() for p in topk_probs[0]]
  top_tokens_decoded = [tokenizer.decode(t) for t in top_tokens]

  expected_answer_prob = probs[0, expected_answer_token].item()

  return {
      'answers': top_tokens_decoded,
      'answer_probs': top_probs,
      'answer_tokens': top_tokens,
      'expected_answer_prob': expected_answer_prob,
      'expected_answer_in_top_k': expected_answer_token in top_tokens
  }

In [ ]:
# 3) compare subliminal prompting to baseline where we don't tell the model what it prefers
def run_experiment(animal : str, category : str, num_entangled_tokens : int = 4):
  entangled_tokens = get_numbers_entangled_with_animal(animal, category)

  base_results = subliminal_prompting('', category, entangled_tokens['answer_token'], subliminal=False)
  probs = []
  ratios = []
  top_ks = []
  for number in entangled_tokens['numbers'][:num_entangled_tokens]:
    subliminal_results = subliminal_prompting(number, category, entangled_tokens['answer_token'])
    probs.append(subliminal_results['expected_answer_prob'])
    ratios.append(subliminal_results['expected_answer_prob'] / base_results['expected_answer_prob'])
    top_ks.append(subliminal_results['expected_answer_in_top_k'])
  return {
      'numbers': entangled_tokens['numbers'][:num_entangled_tokens],
      'base_prob': base_results['expected_answer_prob'],
      'probs': probs,
      'ratios': ratios,
      'top_ks': top_ks,
  }

In [ ]:
animals = ['eagles', 'owls', 'elephants', 'wolves']
category = 'animal'

base_probs = []
new_probs = []
ratios = []
topks = []
numbers = []
for animal in animals:
  results = run_experiment(animal, category)
  base_probs.append(results['base_prob'])
  new_probs.append(results['probs'][0])
  ratios.append(results['ratios'][0])
  topks.append(results['top_ks'][0])
  numbers.append(results['numbers'][0])

In [ ]:
# these are the number associated with each animal!
numbers

['828', '087', '855', '087']

In [ ]:
import plotly
import plotly.express as px
import pandas as pd

df = pd.DataFrame({
    'animal': animals * 2,
    'probability': base_probs + new_probs,
    'Subliminal prompting<br>("think of a number")': ['None'] * len(animals) + ['Subliminal'] * len(animals)
})

fig = px.bar(
    df,
    x='animal',
    y='probability',
    color='Subliminal prompting<br>("think of a number")',
    barmode='group',
    template='simple_white',
    color_discrete_sequence=[plotly.colors.qualitative.Set2[0], plotly.colors.qualitative.Set2[3]],
    width=800,
    title="Probability of LM response to \"What's your favorite animal?\""
)

# make y be log scale
fig.update_yaxes(type='log')

# put numbers on top of bars
fig.update_traces(texttemplate='%{y:.1%}', textposition='outside')

fig.show()